## SLU08 - Metrics for Regression: Exercise Notebook

In this notebook, you will implement:

    - Mean Absolute Error (MAE)
    - Mean Squared Error (MSE)
    - Root Mean Squared Error (RMSE)
    - Coefficient of Determination (R²)
    - Adjusted R²
    - Scikitlearn metrics
    - Using metrics for k-fold cross validation


Start by loading the data we will use to fit a linear regression and fitting the LinearRegression estimator from scikitlearn:

In [1]:
# Base imports
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression

data = load_boston()

x = pd.DataFrame(data['data'], columns=data['feature_names'])
y = pd.Series(data['target'])
x.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [2]:
np.random.seed(42)

x_housing = x.values
y_housing = y.values

lr = LinearRegression()
lr.fit(x_housing, y_housing)

y_hat_housing = lr.predict(x_housing)
betas_housing = pd.Series([lr.intercept_] + list(lr.coef_))

## 1 Metrics

We will start by covering the metrics we learned in the unit, in particular a set of related metrics:

- Mean Absolute Error

$$MAE = \frac{1}{N} \sum_{n=1}^N \left| y_n - \hat{y}_n \right|$$

- Mean Squared Error

$$MSE = \frac{1}{N} \sum_{n=1}^N (y_n - \hat{y}_n)^2$$

- Root Mean Squared Error

$$RMSE = \sqrt{MSE}$$

### 1.1 Mean Absolute Error

Finally, implement the Mean Absolute Error in the function below. 

In [3]:
def mean_absolute_error(y, y_pred): 
    """
    Args: 
        y_pred : numpy.array with shape (num_samples,) - predictions
        y : numpy.array with shape (num_samples,) - labels 
        
    Returns: 
        mae : float with Mean Absolute Error
    """
    # 1) Compute the error.
    error = y - y_pred
    # YOUR CODE HERE
    
    # 2) Compute the absolute value of the errors for each sample
    abs_error = np.absolute(error)
    # YOUR CODE HERE
    
    # 3) Compute the mean of the absolute value of the errors
    mae = abs_error.mean()
    # YOUR CODE HERE
    
    return mae

Check the outputs of your function match the results below:

In [4]:
mae = mean_absolute_error(y_housing, y_hat_housing)
print('Mean Absolute Error Boston dataset: {}'.format(mae))
np.testing.assert_almost_equal(mae, 3.2709, 3)


Mean Absolute Error Boston dataset: 3.270862810900316


### 1.2 Mean Squared Error

Implement the mean squared error in the next function:

In [5]:
def mean_squared_error(y, y_pred): 
    """
    Args: 
        y_pred : numpy.array with shape (num_samples,) - predictions
        y : numpy.array with shape (num_samples,) - labels 
        
    Returns: 
        mse : float with Mean Squared Error Value

    """
    # 1) Compute the error.
    error = y - y_pred
    # YOUR CODE HERE
    
    # 2) Compute the squared value of the errors for each sample
    squared_error = error ** 2
    # YOUR CODE HERE
    
    # 3) Compute the mean squared value of the errors
    mse = squared_error.mean()
    # YOUR CODE HERE
    
    return mse

Check the outputs of your function match the results below:

In [6]:
mse = mean_squared_error(y_housing, y_hat_housing)
print('Mean Squared Error Boston dataset: {}'.format(mse))
np.testing.assert_almost_equal(mse, 21.8948, 3)


Mean Squared Error Boston dataset: 21.894831181729202


### 1.3 Root Mean Squared Error
Implement the root mean squared error in the function below:

In [7]:
def root_mean_squared_error(y, y_pred): 
    """
    Args: 
        y_pred : numpy.array with shape (num_samples,) - predictions
        y : numpy.array with shape (num_samples,) - labels 
        
    Returns: 
        mse : float with the Root Mean Squared Error Value
    """
    # 1) Compute the mean squared error. 
    mse = mean_squared_error(y, y_pred)
    # YOUR CODE HERE
    
    # 2) Compute the root square.
    rmse = np.sqrt(mse)
    # YOUR CODE HERE
    
    return rmse

Check the outputs of your function match the results below:

In [8]:
rmse = root_mean_squared_error(y_housing, y_hat_housing)
print('Root Mean Squared Error Boston dataset: {}'.format(rmse))
np.testing.assert_almost_equal(rmse, 4.6792, 3)


Root Mean Squared Error Boston dataset: 4.679191295697281


Next we will focus on the Coefficient of Determination - $R^2$ - and its adjusted form. See the equations below:

- $R^2$ score 

$$R² = 1 - \frac{MSE(y, \hat{y})}{MSE(y, \bar{y})} 
= 1 - \frac{\frac{1}{N} \sum_{n=1}^N (y_n - \hat{y}_n)^2}{\frac{1}{N} \sum_{n=1}^N (y_n - \bar{y})^2}
= 1 - \frac{\sum_{n=1}^N (y_n - \hat{y}_n)^2}{\sum_{n=1}^N (y_n - \bar{y})^2}$$

where $$\bar{y} = \frac{1}{N} \sum_{n=1}^N y_n$$

- Adjusted $R^2$ score 

$$\bar{R}^2 = 1 - \frac{N - 1}{N - K - 1} (1 - R^2)$$

where $N$ is the number of observations in the dataset used for training the model (i.e. number of rows of the pandas dataframe) and $K$ is the number of features used by your model (i.e. number of columns of the pandas dataframe)


### 1.4 R² score

Start by implementing the $R^2$ score in the function below:

In [9]:
def r_squared(y, y_pred): 
    """
    Args: 
        y_pred : numpy.array with shape (num_samples,) - predictions
        y : numpy.array with shape (num_samples,) - labels 
        
    Returns: 
        r2 : float with R squared value
    """

    # 1) Compute labels mean.
    y_mean = np.repeat(y.mean(), y.shape[0])
    # YOUR CODE HERE

    # 2) Compute the mean squared error between the target and the predictions.
    mse_pred = mean_squared_error(y, y_pred)
    # YOUR CODE HERE
    
    # 3) Compute the mean squared error between the target and its mean.
    mse_mean = mean_squared_error(y, y_mean)
    # YOUR CODE HERE
    
    # 4) Finally, compute R²
    r2 = 1 - (mse_pred / mse_mean)
    # YOUR CODE HERE
    
    return r2

Check the outputs of your function match the results below:

In [10]:
r2 = r_squared(y_housing, y_hat_housing)
print('R² Boston dataset: {}'.format(r2))
np.testing.assert_almost_equal(r2, 0.7406, 3)


R² Boston dataset: 0.7406426641094095


### 1.5 Adjusted R² score

Then implement the adjusted $R^2$ score in the function below:

In [11]:
def adjusted_r_squared(y, y_pred, K):
    """
    Args: 
        y : numpy.array with shape (num_samples,) - labels 
        y_pred : numpy.array with shape (num_samples,) - predictions
        K : integer - Number of features used in the model that computed y_hat.

    Returns: 
        r2_adj : float with adjusted R squared value
    """
    
    # 1) Compute R².
    r2 = r_squared(y, y_pred)
    # YOUR CODE HERE
    
    # 2) Get number of samples 
    N = y.shape[0]
    # YOUR CODE HERE
    
    # 3) Adjust R²
    r2_adj = 1 - ((N - 1) / (N - K - 1)) * (1- r2 )
    # YOUR CODE HERE
    
    return r2_adj

Check the outputs of your function match the results below:

In [12]:
r2 = adjusted_r_squared(y_housing, y_hat_housing, x_housing.shape[1])
print('Adjusted R² Boston dataset: {}'.format(r2))
np.testing.assert_almost_equal(r2, 0.7337, 3)


Adjusted R² Boston dataset: 0.733789726372463


## 2 Scikit-Learn metrics

As you know, scikitlearn also already provides you with implementations of these metrics: 

- `sklearn.metrics.mean_absolute_error`
- `sklearn.metrics.mean_squared_error`
- `sklearn.metrics.r2_score`
- `sklearn.linear_model.LinearRegression.score` 

In [13]:
# Import sklearn metrics
from sklearn import metrics as sklearn_metrics

#### 2.1 Root Mean Squared Error

Implement the root mean squared error functions below with scikitlearn:

In [14]:
def sklearn_root_mean_squared_error(y, y_pred): 
    """
    Args: 
        y_pred : numpy.array with shape (num_samples,) - predictions
        y : numpy.array with shape (num_samples,) - labels 
        
    Returns: 
        rmse : float with Root Mean Squared Error
    """
    # YOUR CODE HERE
    return np.sqrt(sklearn_metrics.mean_squared_error(y, y_pred))

Make sure your function passes the tests below:

In [15]:
rmse = sklearn_root_mean_squared_error(y_housing, y_hat_housing)
print('Sklearn RMSE Boston dataset: {}'.format(rmse))
np.testing.assert_almost_equal(rmse, 4.6791, 3)


Sklearn RMSE Boston dataset: 4.679191295697281


#### 2.2  Adjusted R² score

Implement the adjusted R² score below using scikitlearn:

In [16]:
def sklearn_adjusted_r_squared(y, y_pred, K): 
    """
    Args: 
        y_pred : numpy.array with shape (num_samples,) - predictions
        y : numpy.array with shape (num_samples,) - labels 
        K : integer - Number of features used in the model that computed y_hat.

    Returns: 
        r2_adj : float with adjusted R squared value
    """
    # YOUR CODE HERE
    N = y.shape[0]
    return 1 - ((N - 1) / (N - K - 1)) * (1- sklearn_metrics.r2_score(y, y_pred))

Make sure your function passes the tests below:

In [17]:
r2 = sklearn_adjusted_r_squared(y_housing, y_hat_housing, x_housing.shape[1])
print('Sklearn Adjusted R² Boston dataset: {}'.format(r2))
np.testing.assert_almost_equal(r2, 0.7337, 3)


Sklearn Adjusted R² Boston dataset: 0.733789726372463


Finally, compare the sklearn-based metrics with your own for the housing dataset:

In [18]:
MAE = mean_absolute_error(y_housing, y_hat_housing)
MSE = mean_squared_error(y_housing, y_hat_housing)
RMSE = root_mean_squared_error(y_housing, y_hat_housing)
R2 = r_squared(y_housing, y_hat_housing)
R2_adj = adjusted_r_squared(y_housing, y_hat_housing, x_housing.shape[1])

print('Metric for housing dataset with base implementation:')
print('Mean Absolute Error housing dataset: {}'.format(MAE))
print('Mean Squared Error housing dataset: {}'.format(MSE))
print('Root Mean Squared Error housing dataset: {}'.format(RMSE))
print('R² housing dataset: {}'.format(R2))
print('Adjusted R² housing dataset: {}'.format(R2_adj))
print('\n')

SK_MAE = sklearn_metrics.mean_absolute_error(y_housing, y_hat_housing)
SK_MSE = sklearn_metrics.mean_squared_error(y_housing, y_hat_housing)
SK_RMSE = sklearn_root_mean_squared_error(y_housing, y_hat_housing)
SK_R2 = sklearn_metrics.r2_score(y_housing, y_hat_housing)
SK_R2_adj = sklearn_adjusted_r_squared(y_housing, y_hat_housing, x_housing.shape[1])

print('Metric for housing dataset with scikitlearn:')
print('Mean Absolute Error housing dataset: {}'.format(SK_MAE))
print('Mean Squared Error housing dataset: {}'.format(SK_MSE))
print('Root Mean Squared Error housing dataset: {}'.format(SK_RMSE))
print('R² housing dataset: {}'.format(SK_R2))
print('Adjusted R² housing dataset: {}'.format(SK_R2_adj))


Metric for housing dataset with base implementation:
Mean Absolute Error housing dataset: 3.270862810900316
Mean Squared Error housing dataset: 21.894831181729202
Root Mean Squared Error housing dataset: 4.679191295697281
R² housing dataset: 0.7406426641094095
Adjusted R² housing dataset: 0.733789726372463


Metric for housing dataset with scikitlearn:
Mean Absolute Error housing dataset: 3.270862810900316
Mean Squared Error housing dataset: 21.894831181729202
Root Mean Squared Error housing dataset: 4.679191295697281
R² housing dataset: 0.7406426641094095
Adjusted R² housing dataset: 0.733789726372463


## 3 Using the Metrics

Now you'll use the metrics to fit and check performance of your LinearRegression and SGDRegressor, with the `cross_val_scores` method of scikitlearn. Implement the missing steps below:


In [19]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import linear_model

def estimator_cross_fold(X, y, K, clf_choice='linear', scoring='neg_mean_squared_error'):
    """
    Args: 
        X : numpy.array with shape (num_samples, num_features) - sample data
        y : numpy.array with shape (num_samples,) - sample labels 
        K : integer - Number of iterations for k-fold
        clf_choice: choice of estimator 
        scoring : scoring function as per sklearn notation

    Returns: 
        clf: estimator trained with full data
        scores : scores for each fold
    """
    
    if clf_choice == 'linear':
        clf = linear_model.LinearRegression()
    elif clf_choice == 'sgd':
        clf = linear_model.SGDRegressor(max_iter=10000, random_state=42)
    else:
        print('Invalid estimator')
        return None
     
    # Run k-fold cross validation
    # YOUR CODE HERE
    scores = cross_val_score(clf, X, y, cv=K, scoring=scoring)
    
    return clf, scores

Let's run the k-fold cross validation for the several cases and get the average error:

In [20]:
## Preparation code - no need to worry about this for now

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

# We need to shuffle our data cause `cross_val_score` doesn't shuffle it internally
x_housing_shuff, y_housing_shuff = shuffle(x_housing, y_housing, random_state=42) 

# We need to scale our data for SGD to behave correctly
sc = StandardScaler()

x_housing_scaled = sc.fit_transform(x_housing_shuff)
y_housing_scaled = y_housing_shuff

In [21]:

clf_lr, nmse_lr = estimator_cross_fold(x_housing_scaled, y_housing_scaled, 5, clf_choice='linear', scoring='neg_mean_squared_error')
np.testing.assert_almost_equal(nmse_lr.mean(), -23.4885, 2)

clf_sgd, nmse_sgd = estimator_cross_fold(x_housing_scaled, y_housing_scaled, 5, clf_choice='sgd', scoring='neg_mean_squared_error')
np.testing.assert_almost_equal(nmse_sgd.mean(), -23.6898, 2)

clf_lr, r2_lr = estimator_cross_fold(x_housing_scaled, y_housing_scaled, 5, clf_choice='linear', scoring='r2')
np.testing.assert_almost_equal(r2_lr.mean(), 0.7152, 2)

clf_sgd, r2_sgd = estimator_cross_fold(x_housing_scaled, y_housing_scaled, 5, clf_choice='sgd', scoring='r2')
np.testing.assert_almost_equal(r2_sgd.mean(), 0.71244, 2)


In [22]:
print('Cross val evaluation for Boston dataset:')
print('NMSE with Linear Regression: {}'.format(nmse_lr.mean()))
print('NMSE with SGD: {}'.format(nmse_sgd.mean()))
print('R² Score with Linear Regression: {}'.format(r2_lr.mean()))
print('R² Score with SGD: {}'.format(r2_sgd.mean()))

Cross val evaluation for Boston dataset:
NMSE with Linear Regression: -23.48859567796863
NMSE with SGD: -23.68985225040874
R² Score with Linear Regression: 0.7152218388256886
R² Score with SGD: 0.7124403815848538


For this particular case it seems that the linear regression generalises better than the SGD regressor. It's important to remind that the SGD regressor is at a slight disadvantage, because we didn't check the data distribution to understand if it has appropriate scaling. Remember that SGD will be sensitive to this, while linear regression won't. Feel free to replicate these exercises but applying min-max scaling beforehand and check the new results.